In [1]:
import os
import albumentations as A
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import cv2

from PIL import Image

from albumentations.pytorch import ToTensor
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torch.optim import Adam
from torchvision.models import resnet18
import torch.nn.functional as F

from sklearn.model_selection import train_test_split

import tqdm
import matplotlib.pyplot as plt
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
device = torch.device('cuda')

In [2]:
images_folder_inlines = 'data/train/netherlands/inlines'
images_folder_crosslines = 'data/train/netherlands/crosslines'
masks_folder = 'data/train/netherlands/masks/masks'

In [3]:
df_inlines = pd.DataFrame([[os.path.join(images_folder_inlines, img_name), int(img_name.split('.')[0].split('_')[1]), 'inline'] for img_name in os.listdir(images_folder_inlines)], columns=['image_path', 'id', 'split_type'])
df_crosslines = pd.DataFrame([[os.path.join(images_folder_crosslines, img_name), int(img_name.split('.')[0].split('_')[1]), 'crossline'] for img_name in os.listdir(images_folder_crosslines)], columns=['image_path', 'id', 'split_type'])
df_images = pd.concat((df_inlines, df_crosslines), axis=0).reset_index()
df_masks =  pd.DataFrame([[os.path.join(masks_folder, img_name), int(img_name.split('.')[0].split('_')[1]), img_name.split('.')[0].split('_')[0]] for img_name in os.listdir(masks_folder)], columns=['image_path_masks', 'id', 'split_type'])

In [4]:
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

class ConvRelu(nn.Module):
    def __init__(self, in_, out):
        super().__init__()
        self.conv = conv3x3(in_, out)
        self.activation = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.activation(x)
        return x


class ConvBn2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)):
        super(ConvBn2d, self).__init__()

        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding,
                              bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        # self.bn = SynchronizedBatchNorm2d(out_channels)

    def forward(self, z):
        x = self.conv(z)
        x = self.bn(x)
        return x


class Decoder(nn.Module):
    def __init__(self, in_channels, channels, out_channels):
        super(Decoder, self).__init__()
        self.conv1 = ConvBn2d(in_channels, channels, kernel_size=3, padding=1)
        self.conv2 = ConvBn2d(channels, out_channels, kernel_size=3, padding=1)

    def forward(self, x):
        x = F.upsample(x, scale_factor=2, mode='bilinear')
        x = F.relu(self.conv1(x), inplace=True)
        x = F.relu(self.conv2(x), inplace=True)
        return x


class DecoderSEBlockV2(nn.Module):
    def __init__(self, in_channels, middle_channels, out_channels):
        super().__init__()
        self.in_channels = in_channels
        self.block = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='bilinear'),
            ConvRelu(in_channels, middle_channels),
            ConvRelu(middle_channels, out_channels))
        # SEBlock(planes=out_channels, reduction=16))

    def forward(self, x):
        return self.block(x)

In [5]:
class UnetResnet34(nn.Module):
    def __init__(self, num_classes=1, num_filters=16, pretrained=True):
        super().__init__()
        self.num_classes = num_classes
        self.pool = nn.MaxPool2d(2, 2)
        encoder = resnet18(pretrained=pretrained)
        self.relu = nn.ReLU(inplace=True)

        self.encoder = nn.ModuleList([
            nn.Sequential(
                encoder.conv1,
                encoder.bn1,
                encoder.relu,
                self.pool),
            encoder.layer1,
            encoder.layer2,
            encoder.layer3,
            encoder.layer4])

        self.avgpool = nn.AvgPool2d(3)
        self.fc = nn.Linear(512, 1)

        self.center = DecoderSEBlockV2(512, num_filters * 8 * 2, num_filters * 8)

        self.decoder = nn.ModuleList([
            DecoderSEBlockV2(512 + num_filters * 8, num_filters * 8 * 2, num_filters * 8),
            DecoderSEBlockV2(256 + num_filters * 8, num_filters * 8 * 2, num_filters * 8),
            DecoderSEBlockV2(128 + num_filters * 8, num_filters * 4 * 2, num_filters * 2),
            DecoderSEBlockV2(64 + num_filters * 2, num_filters * 2 * 2, num_filters * 2 * 2),
        ])

        self.dec1 = DecoderSEBlockV2(num_filters * 2 * 2, num_filters * 2 * 2, num_filters)
        self.dec0 = ConvRelu(num_filters, num_filters)
        self.final = nn.Conv2d(num_filters, num_classes, kernel_size=1)

    def forward(self, x):
        encoder_results = []
        for stage in self.encoder:
            x = stage(x)
            encoder_results.append(x.clone())

        x = self.center(self.pool(x))

        for i, decoder in enumerate(self.decoder):
            x = self.decoder[i](torch.cat([x, encoder_results[-i - 1]], 1))

        x = self.dec1(x)
        x = self.dec0(x)
        x = self.final(x)

        return x

In [6]:
class CrossEntropyLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(CrossEntropyLoss, self).__init__()
        self.loss = nn.CrossEntropyLoss(weight, size_average)

    def forward(self, logits, targets):
        targets = targets.type(torch.cuda.LongTensor)
        return self.loss(logits, targets)

    
class MultiDice(nn.Module):
    """
    Calculate Dice with averaging per classes and then per batch 
    """
    def __init__(self,):
        super(MultiDice, self).__init__()

    def forward(self, outputs, targets):
        smooth = 1e-15
        prediction = outputs.softmax(dim=1)
        
        dices = []
        
        for val in range(1, 8):
            target = (targets == val).float().squeeze()
            ch_pred = prediction[:, val]
            intersection = torch.sum(ch_pred * target, dim=(1,2))
            union = torch.sum(ch_pred, dim=(1,2)) + torch.sum(target, dim=(1,2))      
            dice_part = (2 * intersection + smooth) / (union + smooth)
        dices.append(dice_part.mean())
        return torch.mean(dice_part)

In [7]:
train_transforms = A.Compose([
    A.Resize(512, 384),
    A.HorizontalFlip(p=0.5),
    A.Normalize(
        mean=(0.485, 0.456, 0.406),
        std=(0.229, 0.224, 0.225))
    ])

valid_transforms = A.Compose([
    A.Resize(512, 384),
    A.Normalize(
        mean=(0.485, 0.456, 0.406),
        std=(0.229, 0.224, 0.225)),
])

In [8]:
class TrainDataset():
    def __init__(self, images, masks, transform):
        self.masks = masks
        self.images = images
        self.transform = transform

    def __getitem__(self, index):
        path_mask, id_split, split_type = self.masks.loc[index][['image_path_masks', 'id', 'split_type']].values
        path_img = self.images[(id_split == self.images['id']) & (split_type == self.images['split_type'])]['image_path'].values[0]
        
        img = cv2.imread(path_img)
        mask = cv2.imread(path_mask)
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        result = self.transform(
            image=img, 
            mask=mask
        )
        
        result = {
            'image': ToTensor()(image=result['image'])['image'], 
            'mask': torch.Tensor(result['mask'])
        }
        return result
    
    def __len__(self, ):
        return len(self.masks)

In [9]:
df_masks_train, df_masks_valid = train_test_split(df_masks)

In [10]:
train_dataset = TrainDataset(
    images=df_images, 
    masks=df_masks, 
    transform=train_transforms
)

valid_dataset = TrainDataset(
    images=df_images, 
    masks=df_masks,
    transform=valid_transforms
)

In [11]:
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=32, 
    shuffle=True,
)

valid_loader = DataLoader(
    dataset=valid_dataset,
    batch_size=8,                
    shuffle=False,
)

In [12]:
criterion = CrossEntropyLoss()
metric = MultiDice()

/home/core/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [13]:
model = UnetResnet34(
    num_classes=8,
    num_filters=4,
    pretrained=True,
)

model = nn.DataParallel(model).to(device)

optimizer = Adam(model.parameters(), lr=1e-4)
schelduer = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

In [14]:
model.train()
for i, data in enumerate(train_loader):
    inputs = Variable(data['image']).to(device)
    masks = Variable(data['mask']).to(device)
    break

In [15]:
model(inputs).shape

/home/core/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:2494: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


torch.Size([32, 8, 512, 384])

In [16]:
best_model = 0
for epoch in tqdm.tqdm_notebook(range(30)):
    running_loss = 0.0
    value = 0.0
    
    model.train()
    for i, data in enumerate(train_loader):
        inputs = Variable(data['image']).to(device)
        masks = Variable(data['mask']).to(device)
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, masks)
        metric_value = metric(outputs, masks) 
        
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        value += metric_value.item()                    
        print(f"[TRAIN][BATCH {i}/{len(train_loader)}] metric: {value / (i + 1)} loss: {running_loss / (i + 1)}\r", end="")   
        
    running_loss = 0.0
    value = 0.0
    model.eval()
    for i, data in enumerate(valid_loader):
        inputs = Variable(data['image']).to(device)
        masks = Variable(data['mask']).to(device)

        outputs = model(inputs)
        loss = criterion(outputs, masks)
        metric_value = metric(outputs, masks) 
        
        running_loss += loss.item()
        value += metric_value.item()                   
        
        print(f"[VALID][BATCH {i}/{len(valid_loader)}] metric: {value / (i + 1)} loss: {running_loss / (i + 1)}\r", end="")
        
    schelduer.step(running_loss / (i + 1))
    
    if best_model < value:
        torch.save(model.state_dict(), 'best_model.pth')

RuntimeError: cuda runtime error (59) : device-side assert triggered at /opt/conda/conda-bld/pytorch_1573049310284/work/aten/src/THCUNN/generic/SpatialClassNLLCriterion.cu:127